1. Data Structures and Dependency Management

dataclasses is being used for structured data storage and scipy.spatial for efficient distance matrix computation. Each instance encapsulates the spatial coordinates, the corresponding adjacency matrix, and the exact global extrema (optimal and worst-case tour lengths).

In [1]:
import numpy as np
import time
import scipy
from dataclasses import dataclass
from typing import List, Optional, Tuple
from scipy.spatial import distance_matrix

@dataclass
class TSPInstance:
    """
    Data container for a TSP instance and its theoretical bounds.
    
    Attributes:
        kind: The distribution topology ('circle' or 'random').
        N: Number of nodes (cities).
        sigma: Standard deviation of noise applied to circular coordinates.
        coords: (N, 2) array of Cartesian coordinates.
        D: (N, N) Euclidean distance matrix.
        l_star: Optimal (minimal) Hamiltonian cycle length.
        l_worst: Maximal Hamiltonian cycle length.
    """
    kind: str
    N: int
    sigma: Optional[float]
    coords: np.ndarray
    D: np.ndarray
    l_star: float
    l_worst: float

2. Environment Documentation and Reproducibility

In computational research, documenting the software environment is critical for the reproducibility of results. Discrepancies in library versions (e.g., NumPy or SciPy) can lead to subtle variations in floating-point arithmetic or algorithmic behavior.

This section automates the provenance tracking by:

    Generating a requirements.txt file: Capturing the exact state of the pip environment.

    Logging System Metadata: Recording the Python interpreter version, operating system architecture, and core dependency versions.

In [2]:
import sys
import platform
import subprocess

def get_environment_info():
    """
    Captures the runtime environment metadata for scientific documentation.
    Ensures that the exact versions of computational libraries are known.
    """
    info = {
        "python_version": sys.version,
        "platform": platform.platform(),
        "processor": platform.processor(),
        "numpy_version": np.__version__,
        "scipy_version": scipy.__version__
    }
    return info

# Automated generation of requirements.txt for environment replication
with open("requirements_generate_dataset.txt", "w") as f:
    # Captures all installed packages via pip freeze
    subprocess.run(["pip", "freeze"], stdout=f)

# Display and store environment metadata
env_info = get_environment_info()
print("--- Runtime Environment Captured ---")
for key, value in env_info.items():
    print(f"{key}: {value}")

--- Runtime Environment Captured ---
python_version: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:09:02) [GCC 11.2.0]
platform: Linux-6.1.0-38-amd64-x86_64-with-glibc2.36
processor: 
numpy_version: 2.4.2
scipy_version: 1.17.0


3. Compute tour lengths
To establish the ground truth, the Held-Karp algorithm is being used to determine the exact optimal tour length (l_star) and the worst-case tour length (l_worst).

In [3]:
def solve_tsp_extremes(D: np.ndarray) -> Tuple[float, float]:
    """
    Calculates the minimum and maximum Hamiltonian cycles using Dynamic Programming.
    """
    n = len(D)
    memo_min = {}
    memo_max = {}

    def visit(mask, last, find_max=False):
        memo = memo_max if find_max else memo_min
        
        # Termination: All nodes visited; return to origin
        if mask == (1 << n) - 1:
            return D[last][0]
        
        if (mask, last) in memo:
            return memo[(mask, last)]

        if find_max:
            res = max(D[last][i] + visit(mask | (1 << i), i, True)
                      for i in range(n) if not (mask & (1 << i)))
        else:
            res = min(D[last][i] + visit(mask | (1 << i), i, False)
                      for i in range(n) if not (mask & (1 << i)))

        memo[(mask, last)] = res
        return res

    return visit(1, 0, False), visit(1, 0, True)

4. Coordinate Generation and Topologies
- Perturbed Circular Distribution: Points are distributed along a ring of radius R=N/2π and then Gaussian noise, controlled by σ to vary the difficulty of the manifold, is introduced.
- Uniform Random Distribution: Points are sampled from a uniform distribution U(0,1) in a 2D unit square.

In [4]:
def generate_cities(N: int, sigma: float, rng: np.random.Generator) -> Tuple[np.ndarray, np.ndarray]:
    """Generates a circular distribution of cities with additive noise."""
    R = N / (2 * np.pi)
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False)

    rand_phi = rng.uniform(0, 2 * np.pi, N)
    rand_r = rng.uniform(0, sigma, N)

    coords = np.zeros((N, 2))
    coords[:, 0] = R * np.cos(angles) + rand_r * np.cos(rand_phi)
    coords[:, 1] = R * np.sin(angles) + rand_r * np.sin(rand_phi)

    D = distance_matrix(coords, coords)
    return coords, D

def generate_random_cities(N: int, rng: np.random.Generator) -> Tuple[np.ndarray, np.ndarray]:
    """Generates cities within a uniform unit-square distribution."""
    coords = rng.random((N, 2))
    D = distance_matrix(coords, coords)
    return coords, D

5. Complexity Filtering via Greedy Heuristics

To ensure the dataset consists of non-trivial instances, a Greedy Filter is applied. An instance is discarded if a simple Nearest Neighbor (Greedy) heuristic, starting from any node, can achieve the optimal solution l_star.

Selection Criterion: Only "hard" instances where Greedy(D,s)>l∗ for all starting nodes s ∈ {1,…,N} are retained.

In [5]:
def fast_greedy(D: np.ndarray, start: int) -> float:
    """Computes the TSP tour length using the Nearest Neighbor heuristic."""
    n = D.shape[0]
    visited = np.zeros(n, dtype=bool)
    visited[start] = True
    cur = start
    tour_len = 0.0

    for _ in range(n - 1):
        dist_to_others = D[cur].copy()
        dist_to_others[visited] = np.inf
        nxt = np.argmin(dist_to_others)
        tour_len += D[cur, nxt]
        visited[nxt] = True
        cur = nxt

    return tour_len + D[cur, start]

def greedy_filter(D: np.ndarray, l_star: float, atol: float = 1e-7) -> bool:
    """Returns True if the instance is computationally non-trivial for greedy solvers."""
    n = D.shape[0]
    for s in range(n):
        if abs(fast_greedy(D, s) - l_star) < atol:
            return False
    return True

6. Automated Dataset Synthesis

The following routine automates the generation, filtering, and validation process across multiple noise levels (σ).

In [6]:
def generate_dataset_per_N(
        N: int,
        sigmas: Tuple[float, float, float] = (0.6, 1.0, 1.4),
        n_circle_total: int = 150,
        n_random: int = 50,
        seed: int = 42,
        max_trials_per_bucket: int = 100000
) -> List[TSPInstance]:
    """Synthesizes a balanced dataset of filtered TSP instances."""
    per_sigma = n_circle_total // len(sigmas)
    rng = np.random.default_rng(seed)
    dataset: List[TSPInstance] = []

    # Process Circle-based instances
    for sigma in sigmas:
        kept = 0
        trials = 0
        while kept < per_sigma:
            trials += 1
            if trials > max_trials_per_bucket:
                raise RuntimeError(f"Convergence failure: max trials exceeded for sigma={sigma}")

            coords, D = generate_cities(N, sigma=sigma, rng=rng)
            l_star, l_worst = solve_tsp_extremes(D)

            if greedy_filter(D, l_star):
                inst = TSPInstance("circle", N, sigma, coords, D, l_star, l_worst)
                dataset.append(inst)
                kept += 1

    # Process Uniform Random instances
    kept = 0
    while kept < n_random:
        coords, D = generate_random_cities(N, rng)
        l_star, l_worst = solve_tsp_extremes(D)

        if greedy_filter(D, l_star):
            inst = TSPInstance("random", N, None, coords, D, l_star, l_worst)
            dataset.append(inst)
            kept += 1

    return dataset

7. Serialization and Output

The final dataset is serialized into a compressed NumPy format (.npz) for efficient storage and downstream analysis.

In [8]:
import os
from typing import List

def save_dataset_npz(filename: str, dataset: List[TSPInstance], env_info: dict) -> None:
    """
    Ensures the output directory exists and serializes the TSP instances
    to a compressed archive.
    """
    target_dir = "Generated_datasets"

    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Created directory: {target_dir}")

    full_path = os.path.join(target_dir, filename)

    # Serialize environmental metadata for reproducibility
    env_string = str(env_info)

    np.savez_compressed(
        full_path,
        kinds=np.array([inst.kind for inst in dataset]),
        Ns=np.array([inst.N for inst in dataset]),
        sigmas=np.array([inst.sigma if inst.sigma is not None else -1.0 for inst in dataset]),
        coords=np.array([inst.coords for inst in dataset]),
        distances=np.array([inst.D for inst in dataset]),
        l_stars=np.array([inst.l_star for inst in dataset]),
        l_worsts=np.array([inst.l_worst for inst in dataset]),
        metadata=np.array([env_string])
    )
    print(f"Dataset successfully saved to: {full_path}")

In [ ]:
if __name__ == "__main__":
    num_cities = 8
    seed = 42
    #start_t = time.perf_counter()

    print(f"Executing generation for N={num_cities}...")
    dataset = generate_dataset_per_N(N=num_cities, seed=seed)

    filename = f"tsp_benchmark_{num_cities}_cities_seed_{seed}.npz"
    save_dataset_npz(filename, dataset, env_info)

    #execution_time = time.perf_counter() - start_t
    #print(f"Total Execution Time: {execution_time:.2f} seconds")

Executing generation for N=8...


Note on Loading and Inspecting Compressed Datasets

Once the TSP instances and environment metadata are serialized into a .npz archive, they can be retrieved for analysis or further simulations. The np.load function provides a lazy-loading interface, allowing access to specific arrays by their keys (e.g., distances, l_stars) without loading the entire file into memory at once.

To ensure compatibility with archived metadata, use allow_pickle=True

In [ ]:
file_path = os.path.join("Generated_datasets", "tsp_benchmark_9_cities_seed_42.npz")

with np.load(file_path, allow_pickle=True) as data:
    # 1. List all available keys in the archive
    print("Available keys:", data.files)

    # 2. Access  TSP instances data
    distances = data['distances']
    l_stars = data['l_stars']
    coords = data['coords']

    # 3. Retrieve and print environmental metadata
    # (Since it was saved as a single-element array, access index 0)
    metadata = data['metadata'][0]

    print(f"\nSuccessfully loaded {len(distances)} instances.")
    print(f"Optimal lengths (l_stars) for first 3 instances: {l_stars[:3]}")
    print("-" * 30)
    print(f"Environment Metadata:\n{metadata}")